# Tutorial 3:  Basic Usage of the Quantile Conditional Randomization Test

This tutorial demonstrates a basic usage example of the Quantile Conditional Randomization Test (QCRT) to estimate quantiles of individual effects in a partially linear model. 

In [1]:
set.seed(2022)

## Load a toy example data set

In [ ]:
suppressMessages(library(tidyverse))
source("../i-modelx/qcrt.R")

# Load the simulated data set
ifile <- "data/data_n2000_p20_a400-0_i2_delta5_linear_glmnet_b1.txt"
data <- read_delim(ifile, delim=" ", col_types=cols())
X <- select(data, starts_with("X_")) %>% data.matrix()
Y <- select(data, starts_with("Y")) %>% data.matrix() %>% as.numeric()
Z <- select(data, starts_with("Z_")) %>% data.matrix()

# Select one explanatory variable of interest
j <- 1

# Load the ground-truth individual treatment effects
ifile <- "data/ite_n2000_p20_a400-0_i2_delta5_linear_glmnet_b1.txt"
ite.data <- read_delim(ifile, delim=" ", col_types=cols()) %>% data.matrix()
ite <- ite.data

# Print basic information about the data
n <- length(Y)
cat(sprintf("This data set contains %d observations of %d binary explanatory variables and %d covariates.\n", n, ncol(X), ncol(Z)))

In [ ]:
colMeans(ite)[1:4]

In [ ]:
hist(Y)

## Step 1: Compute the propensity scores based on the true P(X=1|Z)

In [ ]:
# For this data set, the propensity scores are a function of Z as follows
compute_propensity_score <- function(Z, j) {
    n <- nrow(Z)
    prop_score = Z[,j]
    return(prop_score)
}

propensity_scores = compute_propensity_score(Z,j)
hist(propensity_scores)

## Step 2: Define the hypothesis to be tested

We will test whether the 90% largest individual effect is smaller than -1.

In [ ]:
k <- round(0.9*n)   # Largest 90% quantile of individual effects
delta <- -1         # Constant threshold
tail <- "right"     # Right-tailed test

hyp.check = check_hypothesis(t(ite), j, k, delta, tail, verbose=TRUE)

## Step 3: Compute a QCRT p-value

In [ ]:
X.all <- cbind(X, Z)
compute_pval(X.all, Y, j, propensity_scores, k, delta, multivariate=TRUE, tail=tail, reference="auto", K=1000)